bing翻译post方法抓包
bing 
Request URL: https://cn.bing.com/ttranslatev3?isVertical=1&&IG=A45D80446A634A558A94AE850C68E671&IID=translator.5028.1
Request Method: POST
Status Code: 200 
Remote Address: 202.89.233.101:443
Referrer Policy: origin-when-cross-origin


quest URL: https://cn.bing.com/ttranslatev3?isVertical=1&&IG=A45D80446A634A558A94AE850C68E671&IID=translator.5028.1
Request Method: POST
Status Code: 200 
Remote Address: 202.89.233.101:443
Referrer Policy: origin-when-cross-origin


FROM DATA
fromLang: auto-detect
text: dhu
to: zh-Hans

In [48]:
# -*- encoding=UTF-8
import re, json, random
import requests
from goose3 import Goose
from goose3.text import StopWordsChinese #导入停用词
from bs4 import BeautifulSoup

### 参考
[注释风格](https://zh-google-styleguide.readthedocs.io/en/latest/google-python-styleguide/python_style_rules/)
[翻译api](https://juejin.im/post/5beaac9cf265da614a3a09a9)
[goose提取正文](https://pypi.org/project/goose3/)
[入门笔记](https://zhuanlan.zhihu.com/c_1231299003028840448)

可拓展范围：

完善异常捕获

目标字数 (调整gene_shuffle_article函数中para_num=5参数)

使用更好的选择方式，比如使得开头段总是在前、结尾段总是在后

选择中间语言（对使用不同的语言的目标用户泛化性能更强，同时需要调整停用词）

混淆程度  （增加互译次数和翻译软件数）

添加搜索和翻译引擎（百度、谷歌搜索，bing、deepl翻译）

若干页搜索的解决（我没解决了）

添加代理ip轮换？ （爬取频率应该不高，好像不是很需要）


In [49]:
def get_access_result(target_url="https://cn.bing.com/", search_word=None):
    """
        Args:
            str:输入搜索字符串

        Returns:
            res:requests返回的response对象

        Raises:
            statuserror:爬取失败

    """
    args={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}
    url=target_url
    search={}
    if search_word is not None:
        search['q']=search_word
    try:
        res=requests.get(url, headers=args, params=search, timeout=10) #设置访问时间，超时便访问失败
        res.raise_for_status()
        res.encoding=res.apparent_encoding
        # print(res.text[:1000])
        return res
    # except TimeoutError as tout:
    #     print('Can\'t access to website:'+target_url)
    #     return None
    except Exception as e:
        print(e)
        print("spider failed")
        return None

In [50]:

class Ciba:
    '''使用金山词霸翻译
        
    Attributes:
        url: 金山词霸post方法的url
        headers: 报头参数

    '''

    def __init__(self):
        # self.word = word
        self.url = 'http://fy.iciba.com/ajax.php?a=fy'
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) '
                          'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36'
        }
        # 构造post请求的参数
        # self.payload = {
        #     'f': from_lang,
        #     't': to_lang,
        #     'w': self.word
        # }

    # 发送请求
    def request_post(self, word, from_lang='auto', to_lang='auto'):
        """
            Args:
                word:翻译内容
                from_lang：源语言
                to_lang：目标语言

            return：
                None：访问失败
                res.content.decode()：解码后的文本

            Raises：
                statuserror:爬取失败            
        """
        payload = {
            'f': from_lang,
            't': to_lang,
            'w': word
        }
        try:
            res = requests.post(url=self.url, headers=self.headers, data=payload)
            res.raise_for_status()
            # print(res.content.decode())
            return res.content.decode()
        except Exception as e:
            print(e)
            return None
    # 解析数据
    @staticmethod
    def parse_data(data):
        """
            Args：
                data：爬虫解码后的文本

            return：
                dict_data['content']['out']:文字段翻译结果
                dict_data['content']['word_mean']：词语翻译结果
        """
        dict_data = json.loads(data)  #使用json解析
        if 'out' in dict_data['content']:   #文字段翻译
            return dict_data['content']['out']
        elif 'word_mean' in dict_data['content']:  #词语翻译
            return dict_data['content']['word_mean']

    def translate(self, word, from_lang='auto', to_lang='auto'):
        """
            Args:
                word:翻译内容
                from_lang：源语言
                to_lang：目标语言

            return：
                None：访问失败
                res.content.decode()：解码后的文本           
        """
        data = self.request_post(word,from_lang,to_lang)
        return self.parse_data(data)


In [51]:
class bing_engine:
    '''使用bing搜索
        
    Attributes:
        url: bing搜索根目录url
        headers: 报头参数


    ''' 

    def __init__(self):
        self.url="https://bing.com/"
        self.headers={'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}

    def search(self, search_word=None):
        """
            Args:
                search_word:输入搜索字符串

            Returns:
                clean_res:返回一组搜索结果和链接

            Raises:
            statuserror:爬取失败

        """
        search_para={}
        if str is not None:
            search_para['q']=search_word
        try:
            res=requests.get(url=self.url, headers=self.headers, params=search_para)
            res.raise_for_status()
            res.encoding=res.apparent_encoding
            # print(res.text[:1000])
            soup=BeautifulSoup(res.text, 'lxml')
            clean_res=[]
            for i in soup.findAll(name=['a'], target='_blank', text=re.compile('[^帮助]')):
                # print(i.string,'\n',i.attrs['href'])
                clean_res.append((i.string, i.attrs['href']))
            return clean_res
        except Exception as e:
            print(e)
            print("spider failed")
            return None

In [52]:
def get_paragrams(search_res):
    """
        TODO
    """
    paras=[]
    goose = Goose({'browser_user_agent': 'Mozilla', 'parser_class':'soup','stopwords_class': StopWordsChinese})  # 设置goose参数
    for  ind, res_elem in enumerate(search_res):
        res_herf=res_elem[1]
        if get_access_result(target_url=res_herf)==None:  #测试是否可以访问
            print('Can\'t access to website:'+res_herf)
            continue
        article=goose.extract(url=res_herf)  #正文提取
        paras.extend(list(article.cleaned_text.split()))  #分割成段
    return paras

In [53]:
def gene_shuffle_article(search_res, para_num=5):
    """
        TODO
    """
    paragrams=get_paragrams(search_res)  # 获取段落
    # print(len(paragrams))
    total=len(paragrams) # 总段落
    article_gene=''
    for i in random.sample(range(total),para_num):  #生成不重复的随机数
        # print(i, '\n' , paragrams[i])
        article_gene+=paragrams[i]+'\n'  # 选择若干段落拼接
    return article_gene

In [54]:
def rubbish_essay(search_word,search_engine='bing',trans_engine='ciba'):
    """
        TODO
    """
    if search_engine=='bing':
        s_engine=bing_engine()
    search_res=s_engine.search(search_word)
    # print(search_res)
    article_gene=gene_shuffle_article(search_res)
    if trans_engine=='ciba':
        t_engine=Ciba()
    article_en=t_engine.translate(word=article_gene, from_lang='zh',to_lang='en')
    article_final=t_engine.translate(word=article_en, from_lang='en',to_lang='zh')
    return article_final

In [55]:
print(rubbish_essay(search_word="抗疫感想"))

HTTPSConnectionPool(host='www.chinactp.org', port=443): Read timed out. (read timeout=10)
spider failed
Can't access to website:https://www.chinactp.org/wen/fanwen/392184.html
HTTPSConnectionPool(host='www.chinactp.org', port=443): Read timed out. (read timeout=10)
spider failed
Can't access to website:https://www.chinactp.org/wen/fanwen/408095.html
在今年的元旦期间，新的冠状病毒扰乱了我们的新年计划，甚至稀释了一年的味道。 随着疫情的发展和相关的保护措施、法规、文件等。 我成为了一名志愿者，作为一名志愿者，积极参与以上安排的志愿工作，不为钱，不为利，只有面对这一重大疫情贡献自己的微薄之力，使我国顺利度过难关..
 瘟疫前，逆行可哭.. 预防战线，有太多的背影和故事让我们热泪盈眶.. 比如白衣天使们总是奋战在防疫第一线，有的为大家舍小家，有的日夜坚守岗位，有的把休息时间交给同事..比如职工自发集结，奋战72小时，让老楼变成新医院，以备救治和抢救需要；比如党员干部以冲锋队的姿态，保障群众的健康安全.. 党员带头当先锋，医务人员自愿，爱心人士积极参与.. 他们有的告别了刚断奶的孩子，有的离开了新婚夫妇，有的夫妻同时打架，有的全家人，有的累得直起腰来。 他们用辛勤的劳动和精湛的技术，筑牢了防疫防线！！ 抗击疫情的斗争还在继续，战争“疫情”还远未结束，对于一线“战士”来说，他们奋斗的样子值得铭记，奉献的精神值得致敬，疲惫的身心也需要细心的呵护！
  “初心”耀耀，党旗飘扬，战斗堡垒连成一片，铸就“红色长城”。 面对疫情，在短时间内，国道进出口，高铁站出口和重要高速公路卡口悬挂党旗，交通堵塞防控的临时党支部如雨后春笋.. 同时，在乡村街道，村舍，居民楼，一处疫情防控巡逻临时党支部分散.. 一批党员志愿者带领群众参与了疫情防控宣传，重点人员处理，物资协调供应，后勤保障服务等工作.. 新型冠状病毒肺炎爆发以来，我们在全国各地看到，各级党

In [56]:
'''
test
'''
bing=bing_engine()
bing.search("抗疫感想") 

[('抗疫心得体会范文大全【10篇】', 'https://m.51test.net/show/9848999.html'),
 ('抗疫心得_抗击疫情心得体会范文6篇-理财-钱来也', 'https://www.qianlaiye.com/news/99718.html'),
 ('2020普通党员抗疫心得体会范文5篇', 'https://www.chinactp.org/wen/fanwen/392184.html'),
 ('2020青年抗疫心得体会_绽放战疫青春·坚定制度自信',
  'http://m.xuexi.la/xindetihui/fanwen/263038.html'),
 ('抗疫心得体会', 'https://www.jxscct.com/fw/52724/'),
 ('抗疫心得体会1000字精选三篇', 'http://www.lizhigushi.com/renshengganwu/a21428.html'),
 ('抗疫志愿者的心得体会 - 心得体会 - 复来作文网', 'http://www.fulay.cn/html/105-176/176319.htm'),
 ('抗疫作文_感悟英雄在身边', 'https://www.chinactp.org/wen/fanwen/408095.html'),
 ('最新党员抗击疫情工作个人感悟_2020战疫工作心得体会_学习力',
  'http://www.xuexili.com/xindetihui/3254.html'),
 ('新冠肺炎疫情防控心得1000字大全5篇_学习力', 'http://www.xuexili.com/xindetihui/3169.html')]